# Feature Engineering
---


### 1. รหัส ชื่อ และหมู่ของนิสิตในกลุ่ม
+ 6610450960 นันท์นภัส ภูริภัทรพันธุ์ หมู่ 200    
+ 6610450986 พชร พรมแพง หมู่ 200
+ 6610451061 รตนกมล แก้วมาตย์ หมู่ 200

### 2. วัตถุประสงค์ของระบบต้นแบบ
เพื่อแยกรูปภาพว่าถูกสร้างจาก AI-generated หรือมนุษย์ 

เนื่องจากในระยะเวลาไม่กี่ปีที่ผ่านมา AI ได้เข้ามามีบทบาทมากขึ้นในการทำงานส่วนต่าง ๆ รวมถึงได้เกิด AI ที่สามารถสร้างภาพเสมือนจริงขึ้นมาได้ 

ส่งผลกระทบต่ออาชีพของศิลปินและนักออกแบบที่ใช้ทักษะมนุษย์ในการสร้างสรรค์งาน และการสร้างภาพเสมือนจริงโดยใช้ AI ยังก่อให้เกิดปัญหาข่าวลวงและข้อมูลเท็จ ทำให้สร้างความเข้าใจผิดต่อประชาชนได้ ซึ่งเป็นปัญหาสำคัญในปัจจุบัน เช่น ช่างภาพใช้ AI ผลิตภาพจนชนะการประกวดภาพถ่ายของมนุษย์ ข่าวลวงเรื่องสัตว์สายพันธุ์ใหม่ซึ่งเป็นภาพ AI-generated เป็นต้น

ดังนั้นการมีระบบต้นแบบที่สามารถแยกรูปภาพ AI-generated หรือ ภาพที่มนุษย์สร้างขึ้นมา เป็นตัวช่วยในการตรวจสอบ และลดปัญหาเหล่านี้ให้น้อยลง


### 3. ลิงค์ไปยังข้อมูลที่จะใช้ในระบบต้นแบบ
https://www.kaggle.com/datasets/birdy654/cifake-real-and-ai-generated-synthetic-images

### 4. มีค่าที่หายไป (missing values) หรือไม่ อะไรบ้าง แบบใด มากน้อยแค่ไหน แก้ไขได้อย่างไร (ถ้าเป็นรูปภาพ มีรูปที่ไม่ถูกต้องหรือสมบูรณ์หรือไม่ อย่างไร แก้ไขอย่างไร)


เนื่องจากรูปภาพเปิดได้ทุกไฟล์ จึงไม่มี missing value

In [2]:
import os
from PIL import Image

def check_images(directory, corrupted_files):
    for filename in os.listdir(directory):
        if filename.endswith('.jpg'):
            try:
                img = Image.open(os.path.join(directory, filename))
                img.verify() 
            except (IOError, SyntaxError) as e:
                corrupted_files.append(filename)
    return corrupted_files

corrupted_files = []

corrupted_files = check_images('./train/REAL' ,corrupted_files)
corrupted_files = check_images('./train/FAKE' ,corrupted_files)
corrupted_files = check_images('./test/REAL' ,corrupted_files)
corrupted_files = check_images('./test/FAKE' ,corrupted_files)

print(f'Corrupted files: {corrupted_files}')

Corrupted files: []


### 5. แต่ละคุณลักษณะ/feature ต้องมีการทำกระบวนการต่างๆ (operations) หรือไม่ อะไรบ้าง อย่างไร (ถ้าเป็นรูปภาพ ต้องมีลักษณะอย่างไร เช่น ขนาด ความละเอียด รูปแบบไฟล์ ฯลฯ)


ข้อมูลรูปภาพมีความละเอียด 32x32 pixel มีรูปแบบไฟล์เป็น jpg 

In [9]:
import os

def get_file_sizes(directory):
    sizes = []

    for filename in os.listdir(directory):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            file_path = os.path.join(directory, filename)
            file_size = os.path.getsize(file_path)
            sizes.append(file_size)
    
    return sizes

def calculate_statistics(sizes):
    if not sizes:
        return None, None, None, None

    avg_size = sum(sizes) / len(sizes)
    min_size = min(sizes)
    max_size = max(sizes)

    return avg_size, min_size, max_size

directory = './train/FAKE'  # Change this to your directory path
sizes = get_file_sizes(directory)
avg_size, min_size, max_size = calculate_statistics(sizes)

# Convert sizes to KB and MB for readability
avg_size_kb = avg_size / 1024
min_size_kb = min_size / 1024
max_size_kb = max_size / 1024

avg_size_mb = avg_size_kb / 1024
min_size_mb = min_size_kb / 1024
max_size_mb = max_size_kb / 1024

print(f'Average size: {avg_size:.2f} bytes ({avg_size_kb:.2f} KB, {avg_size_mb:.2f} MB)')
print(f'Minimum size: {min_size:.2f} bytes ({min_size_kb:.2f} KB, {min_size_mb:.2f} MB)')
print(f'Maximum size: {max_size:.2f} bytes ({max_size_kb:.2f} KB, {max_size_mb:.2f} MB)')


Average size: 924.52 bytes (0.90 KB, 0.00 MB)
Minimum size: 693.00 bytes (0.68 KB, 0.00 MB)
Maximum size: 1187.00 bytes (1.16 KB, 0.00 MB)


### 6. สำหรับปัญหาและข้อมูลนี้ สามารถป้องกันการเกิด data leakage ได้อย่างไรบ้าง
Dataset นี้ทางคนสร้างข้อมูลได้มีการมีแบ่งข้อมูลตัวอย่างเป็น Train และ Test ไว้แล้ว อย่างชัดเจน ทำให้ข้อมูลตัวอย่างใน Train และ Test ไม่มีข้อมูลที่ซ้ำกัน และทำ image processing แยกระหว่างข้อมูล Train และ Test เพื่อป้องกันไม่ให้ model เห็นข้อมูล test set ก่อน

### 7. วิเคราะห์ความสำคัญของคุณลักษณะ ตัวไหนควรและไม่ควรใช้ (ถ้าเป็นรูปภาพ ให้วิเคราะห์ลักษณะของรูปที่ควรและไม่ควรใช้)
+ ภาพที่ควรใช้

    1. ภาพสีที่มองเห็นชัดเจน
    
    2. ภาพที่มีจุดสี pixel เพื่อวิเคราะห์ แยกความแตกต่างภาพจริงและภาพสังเคราะห์
	
+ ภาพที่ไม่ควรใช้ 

    1. ภาพขาวดำ หรือมี noise มากจนไม่สามารถระบุได้ 
    
    2. ภาพที่ถูกแก้ไขหรือปรับแต่งอย่างหนัก หรือ มีการใช้ฟิลเตอร์หรือเอฟเฟกต์พิเศษ อาจทำให้โมเดลสับสนและไม่สามารถแยกแยะระหว่างภาพจริงและภาพสังเคราะห์ได้
    
    3. ภาพที่มีวัตถุหรือสิ่งกีดขวางบดบังส่วนที่สำคัญของภาพ อาจทำให้โมเดลไม่สามารถวิเคราะห์ได้อย่างถูกต้อง
	


### 8. ประเด็นอื่น ๆ ที่เกี่ยวข้องกับข้อมูลของตัวเอง
มีการใช้ Pillow (Python Imaging Library) เพื่อตรวจสอบไฟล์เสีย (missing value) ของรูปภาพ

### 9. การมีส่วนร่วมของสมาชิกแต่ละคนในกลุ่ม (แต่ละคนทำอะไรบ้าง)

+ 6610450960 นันท์นภัส ภูริภัทรพันธุ์ หมู่ 200 (วางแผนการทำงาน, หาข้อมูล)
+ 6610450986 พชร พรมแพง หมู่ 200 (วิเคราะห์ข้อมูล, เขียนโค้ด)
+ 6610451061 รตนกมล แก้วมาตย์ หมู่ 200 (เรียบเรียงข้อมูล, เขียนโค้ด)


### 10. การเปิดเผยการใช้เครื่องมือปัญญาประดิษฐ์ (ใช้อะไร ใช้เพื่ออะไร ใช้อย่างไร, prompt อย่างไร)

10.1 ChatGPT ใช้เพื่อหาวิธี validate ข้อมูลภาพว่ามีสูญหายหรือไม่สมบรูณ์อย่างไรบ้าง 
prompt: “How to validation if images in this dataset are completely normal ?”


10.2 ChatGPT ใช้เพื่อค้นหา Range ของขนาดรูปภาพใน folder 
prompt: “Write me a code that find avg, min, max in size of image in bytes”
